In [21]:
import math
import torch
import numpy as np
# import gpytorch
import pandas as pd
import os
import pickle
import shutil

import sklearn
from sklearn.model_selection import KFold

import imblearn as imb
# print("imblearn version: ",imblearn.__version__)
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import ADASYN
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import itertools

from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score

from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from rdkit import Chem
from rdkit.Chem import Draw
import sys
sys.path.append('../')
import utils

from sklearn.model_selection import GridSearchCV
from VisUtils import *
from RF_GSCV import *
from sklearn.model_selection import KFold
from split_data import *
from matplotlib import pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
def scale_data(train_x_df, train_y_df, test_x_df, test_y_df): 
    x_df = pd.concat([train_x_df, test_x_df])
    scaling=StandardScaler()
    scaling.fit(x_df)
    Scaled_data=scaling.transform(x_df)
    train_x = scaling.transform(train_x_df)
    test_x = scaling.transform(test_x_df) 
    
    train_y = train_y_df.to_numpy().flatten()
    test_y = test_y_df.to_numpy().flatten()
    return train_x, train_y, test_x, test_y
    

# Split into train/test splits based on fold 
Here, fold1 is the testset 

In [23]:
smile_prop_df = pd.read_csv('../../../../data/datasets/NEK_data_4Berkeley/NEK2/NEK2_1_uM_min_50_pct_inhibition_5fold_random_imbalanced.csv')
trues1 = smile_prop_df[smile_prop_df['fold']=='fold1']['active'].value_counts()
trues1.value_counts()
trues1
smile_prop_df['active'].value_counts()

active
0    1904
1     140
Name: count, dtype: int64

# Normalizing


In [24]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

def smiles_to_fps(smiles_list, radius=2, nBits=2048):
    fps = []
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        # mol = utils.normalize(mol)
        if mol is not None:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
            arr = np.zeros((1,), dtype=np.int8)
            Chem.DataStructs.ConvertToNumpyArray(fp, arr)
            fps.append(arr)
    return np.array(fps)

In [25]:

# original splits
split_path = '../../../../data/NEK_data_4Berkeley/NEK2/'
train_x_df = pd.read_csv(split_path+"/NEK2_inhibition_random_fold1_trainX.csv")
train_y_df= pd.read_csv(split_path+"/NEK2_inhibition_random_fold1_trainY.csv")
test_x_df= pd.read_csv(split_path+"/NEK2_inhibition_random_fold1_testX.csv")
test_y_df= pd.read_csv(split_path+"/NEK2_inhibition_random_fold1_testY.csv")

split_path = '../../../../data/NEK_data_4Berkeley/NEK2/inhib/'
# train_x_df.to_csv(split_path+'inhib_train_x_scaledoriginal.csv',index=False, header=False)
# train_y_df.to_csv(split_path+'inhib_train_y_scaledoriginal.csv',index=False, header=False)
# test_x_df.to_csv(split_path+'inhib_test_x_scaledoriginal.csv',index=False, header=False)
# test_y_df.to_csv(split_path+'inhib_test_y_scaledoriginal.csv',index=False, header=False)

In [26]:
train_x_df.shape, test_x_df.shape

((1635, 306), (409, 306))

In [27]:
x_inhib = pd.concat([train_x_df, test_x_df])
x_inhib.columns = x_inhib.columns.astype(str)

scaling = StandardScaler()
scaling.fit(x_inhib)

train_x_moe = scaling.transform(train_x_df)
test_x_moe = scaling.transform(test_x_df)
train_y_moe = train_y_df.to_numpy().reshape(-1)
test_y_moe = test_y_df.to_numpy().reshape(-1)

In [28]:
inhib_df = pd.read_csv('../../../../data/datasets/NEK_data_4Berkeley/NEK2/NEK2_1_uM_min_50_pct_inhibition_5fold_random_imbalanced.csv')

inhib_train = inhib_df[inhib_df['fold'] != 'fold1'].drop(columns=['fold'])
inhib_test = inhib_df[inhib_df['fold'] == 'fold1'].drop(columns=['fold'])


trainX_inhib_smiles = inhib_train['base_rdkit_smiles']
testX_inhib_smiles = inhib_test['base_rdkit_smiles']



In [33]:
trainX_inhib_MF = smiles_to_fps(trainX_inhib_smiles, radius=2)
testX_inhib_MF = smiles_to_fps(testX_inhib_smiles, radius=2)


In [30]:

train_x_df = pd.DataFrame(train_x_moe) 
train_y_df = pd.DataFrame(train_y_moe) 
test_y_df = pd.DataFrame(test_y_moe)
test_x_df = pd.DataFrame(test_x_moe)

# train_x_df.to_csv(split_path+'inhib_train_x_normalized.csv', index=False, header=False)
# train_y_df.to_csv(split_path+'inhib_train_y_normalized.csv', index=False, header=False) 
# test_y_df.to_csv(split_path+'inhib_test_y_normalized.csv', index=False, header=False) 
# test_x_df.to_csv(split_path+'inhib_test_x_normalized.csv', index=False, header=False) 
# train_x_df.to_csv(split_path+'inhib_train_x_scaledoriginal.csv',index=False, header=False)
# train_y_df.to_csv(split_path+'inhib_train_y_scaledoriginal.csv',index=False, header=False)
# test_x_df.to_csv(split_path+'inhib_test_x_scaledoriginal.csv',index=False, header=False)
# test_y_df.to_csv(split_path+'inhib_test_y_scaledoriginal.csv',index=False, header=False)

In [6]:
split_path = '../../../../data/NEK_data_4Berkeley/NEK2/inhib/'
trainX_norm = pd.read_csv(split_path+"inhib_train_x_normalized.csv",header=None)
train_y_df= pd.read_csv(split_path+"inhib_train_y_normalized.csv",header=None)
test_x_df= pd.read_csv(split_path+"inhib_test_x_normalized.csv",header=None)
test_y_df= pd.read_csv(split_path+"inhib_test_y_normalized.csv",header=None)

In [7]:
test_y_df.value_counts()

0
0    381
1     28
Name: count, dtype: int64

In [31]:
train_x = trainX_norm.to_numpy()
train_y = train_y_df.to_numpy().reshape(-1)
test_x = test_x_df.to_numpy()
test_y= test_y_df.to_numpy().reshape(-1)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(1635, 306) (1635,) (409, 306) (409,)


# norm SMOTE

In [32]:
# SMOTE that ^ 
oversample = SMOTE()
trainX_normSMOTE_temp, trainy_normSMOTE_temp = oversample.fit_resample(train_x,train_y)
trainX_normSMOTE = trainX_normSMOTE_temp
trainy_normSMOTE = trainy_normSMOTE_temp.reshape(-1)

train_x_dfSMOTE = pd.DataFrame(trainX_normSMOTE) 
train_y_dfSMOTE = pd.DataFrame(trainy_normSMOTE.reshape(-1)) 
test_y_df = pd.DataFrame(test_y)

test_x_df = pd.DataFrame(test_x)
train_x_dfSMOTE.shape, train_y_dfSMOTE.shape, test_y_df.shape, test_x_df.shape

# train_x_dfSMOTE.to_csv(split_path+'inhib_train_x_normSMOTE.csv', index=False, header=False)
# train_y_dfSMOTE.to_csv(split_path+'inhib_train_y_normSMOTE.csv', index=False, header=False) 
# test_y_df.to_csv(split_path+'inhib_test_y_normSMOTE.csv', index=False, header=False) 
# test_x_df.to_csv('inhib_test_x_normSMOTE.csv', index=False, header=False) 

((3046, 306), (3046, 1), (409, 1), (409, 306))

# norm UNDER

In [12]:

undersample = RandomUnderSampler()
train_x_UNDER, train_y_UNDER = undersample.fit_resample(train_x,train_y)

train_y_UNDER = train_y_UNDER.flatten()

train_x_dfUNDER = pd.DataFrame(train_x_UNDER) 
train_y_dfUNDER = pd.DataFrame(train_y_UNDER) 
test_y_df = pd.DataFrame(test_y)
test_x_df = pd.DataFrame(test_x)

# train_x_dfUNDER.to_csv(split_path+'inhib_train_x_normUNDER.csv', index=False, header=False)
# train_y_dfUNDER.to_csv(split_path+'inhib_train_y_normUNDER.csv', index=False, header=False) 
# test_y_df.to_csv(split_path+'inhib_test_y_normUNDER.csv', index=False, header=False) 
# test_x_df.to_csv(split_path+'inhib_test_x_normUNDER.csv', index=False, header=False) 
# print(train_x_dfUNDER.shape, train_y_dfUNDER.shape, test_y_df.shape, test_x_df.shape)

(224, 306) (224, 1) (409, 1) (409, 306)


# norm ADASYN

In [14]:

adasyn = ADASYN() 
train_x_tempADASYN, train_y_tempADASYN = adasyn.fit_resample(train_x, train_y)
train_xADASYN = train_x_tempADASYN
train_yADASYN = train_y_tempADASYN.flatten()

train_xADASYN.shape,train_yADASYN.shape, test_x.shape, test_y.shape

train_x_dfADASYN = pd.DataFrame(train_xADASYN) 
train_y_dfADASYN = pd.DataFrame(train_yADASYN.reshape(-1)) 
test_y_df = pd.DataFrame(test_y)
test_x_df = pd.DataFrame(test_x)

train_x_dfADASYN.to_csv(split_path+'inhib_train_x_normADASYN.csv', index=False, header=False)
train_y_dfADASYN.to_csv(split_path+'inhib_train_y_normADASYN.csv', index=False, header=False) 
test_y_df.to_csv(split_path+'inhib_test_y_normADASYN.csv', index=False, header=False) 
test_x_df.to_csv(split_path+'inhib_test_x_normADASYN.csv', index=False, header=False) 
print(train_x_dfADASYN.shape, train_y_dfADASYN.shape, test_y_df.shape, test_x_df.shape)

(3037, 306) (3037, 1) (409, 1) (409, 306)


# Morgan fingerprints wiht radius 2 
r=2

In [34]:
trainX_inhib_MF = smiles_to_fps(trainX_inhib_smiles, radius=2)
testX_inhib_MF = smiles_to_fps(testX_inhib_smiles, radius=2)
trainY_inhib_MF = inhib_train['active'].to_numpy().reshape(-1)
testY_inhib_MF = inhib_test['active'].to_numpy().reshape(-1)

train_x_mf = pd.DataFrame(trainX_inhib_MF) 
train_y_mf = pd.DataFrame(trainY_inhib_MF) 
test_y_mf = pd.DataFrame(testY_inhib_MF)
test_x_mf = pd.DataFrame(testX_inhib_MF)

train_x_mf.to_csv(split_path+'inhib_train_x_normMorgan2.csv', index=False, header=False)
train_y_mf.to_csv(split_path+'inhib_train_y_normMorgan2.csv', index=False, header=False) 
test_y_mf.to_csv(split_path+'inhib_test_y_normMorgan2.csv', index=False, header=False) 
test_x_mf.to_csv(split_path+'inhib_test_x_normMorgan2.csv', index=False, header=False) 

In [35]:
train_x_mf.shape,train_y_mf.shape, test_x_mf.shape, test_y_mf.shape

((1635, 2048), (1635, 1), (409, 2048), (409, 1))

# Morgan fingerprints + SMOTE 
r = 2

In [36]:
oversample = SMOTE()
trainX_Morgan_SMOTE, trainy_Morgan_SMOTE = oversample.fit_resample(train_x_mf, train_y_mf)
train_x_dfmorganSMOTE = pd.DataFrame(trainX_Morgan_SMOTE) 
train_y_dfmorganSMOTE = pd.DataFrame(trainy_Morgan_SMOTE) 
test_y_dfmorganSMOTE = pd.DataFrame(test_y_mf)
test_x_dfmorganSMOTE = pd.DataFrame(test_x_mf)

train_x_dfmorganSMOTE.to_csv(split_path+'inhib_train_x_Morgan2SMOTE.csv', index=False, header=False)
train_y_dfmorganSMOTE.to_csv(split_path+'inhib_train_y_Morgan2SMOTE.csv', index=False, header=False) 
test_y_dfmorganSMOTE.to_csv(split_path+'inhib_test_y_Morgan2SMOTE.csv', index=False, header=False) 
test_x_dfmorganSMOTE.to_csv(split_path+'inhib_test_x_Morgan2SMOTE.csv', index=False, header=False) 
train_x_dfmorganSMOTE.shape, train_y_dfmorganSMOTE.shape, test_y_dfmorganSMOTE.shape, test_x_dfmorganSMOTE.shape

((3046, 2048), (3046, 1), (409, 1), (409, 2048))

In [ ]:
test_y_dfmorganSMOTE.value_counts()

# Morgan fingerprints + UNDER

In [37]:
undersample = RandomUnderSampler()
train_x_Morgan_UNDER, train_y_Morgan_UNDER = undersample.fit_resample(train_x_mf, train_y_mf)

train_x_dfMorganUNDER = pd.DataFrame(train_x_Morgan_UNDER) 
train_y_dfMorganUNDER = pd.DataFrame(train_y_Morgan_UNDER) 
test_y_dfMorganUnder = pd.DataFrame(test_y_mf)
test_x_dfMorganUnder = pd.DataFrame(test_x_mf)

train_x_dfMorganUNDER.to_csv(split_path+'inhib_train_x_Morgan2UNDER.csv', index=False, header=False)
train_y_dfMorganUNDER.to_csv(split_path+'inhib_train_y_Morgan2UNDER.csv', index=False, header=False) 
test_y_dfMorganUnder.to_csv(split_path+'inhib_test_y_Morgan2UNDER.csv', index=False, header=False) 
test_x_dfMorganUnder.to_csv(split_path+'inhib_test_x_Morgan2UNDER.csv', index=False, header=False) 

In [39]:
print(train_x_dfMorganUNDER.shape, train_y_dfMorganUNDER.shape, test_y_dfMorganUnder.shape,test_x_dfMorganUnder.shape)

(224, 2048) (224, 1) (409, 1) (409, 2048)


# Morgan fingerprints + ADASYN

In [38]:

adasyn = ADASYN() 
train_x_MorganADASYN, train_y_MorganADASYN = adasyn.fit_resample(train_x_mf, train_y_mf)


train_x_dfMorganADASYN = pd.DataFrame(train_x_MorganADASYN) 
train_y_dfMorganADASYN = pd.DataFrame(train_y_MorganADASYN) 
test_y_dfMorganADASYN = pd.DataFrame(test_y_mf)
test_x_dfMorganADASYN = pd.DataFrame(test_x_mf)

train_x_dfMorganADASYN.to_csv(split_path+'inhib_train_x_Morgan2ADASYN.csv', index=False, header=False)
train_y_dfMorganADASYN.to_csv(split_path+'inhib_train_y_Morgan2ADASYN.csv', index=False, header=False) 
test_y_dfMorganADASYN.to_csv(split_path+'inhib_test_y_Morgan2ADASYN.csv', index=False, header=False) 
test_x_dfMorganADASYN.to_csv(split_path+'inhib_test_x_Morgan2ADASYN.csv', index=False, header=False) 

In [40]:
print(train_x_dfMorganADASYN.shape, train_y_dfMorganADASYN.shape, test_y_dfMorganADASYN.shape,test_x_dfMorganADASYN.shape)

(3027, 2048) (3027, 1) (409, 1) (409, 2048)


# Combine MOE with Morgan fingerprints 

In [ ]:
X_train_allfeats = np.hstack((X_existing_train, X_morgan_train))
X_test_allfeats = np.hstack((X_existing_test, X_morgan_test)) # X_test_allfeats.shape (283, 2354) # now 2354 feats
# y_test and y_testmrogan == same 
train_combo = pd.DataFrame(X_train_allfeats) # train X 
test_combo = pd.DataFrame(X_test_allfeats) # test X 
y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_testmorgan)
# train_combo.shape, test_combo.shape, X_existing_train.shape, len(X_morgan_train), X_train_allfeats.shape
# train_combo.to_csv(split_path+'inhib_train_x_MOEMorgan2combo.csv', index=False, header=False)
# y_train_df.to_csv(split_path+'inhib_train_y_MOEMorgan2combo.csv', index=False, header=False)
# y_test_df.to_csv(split_path+'inhib_test_y_MOEMorgan2combo.csv', index=False, header=False)
# test_combo.to_csv(split_path+'inhib_test_x_MOEMorgan2combo.csv', index=False, header=False)

In [ ]:
train_combo.shape, y_train_df.shape, test_combo.shape,y_test_df.shape

# Combo r = 2, SMOTE 

In [ ]:
oversample = SMOTE()
trainX_allSMOTE, trainy_allSMOTE = oversample.fit_resample(X_train_allfeats, y_train)
trainXdfcomboSMOTE = pd.DataFrame(trainX_allSMOTE)
trainydfcomboSMOTE = pd.DataFrame(trainy_allSMOTE)
test_x_comboSMOTE = pd.DataFrame(X_test_allfeats)

# trainXdfcomboSMOTE.to_csv(split_path+'inhib_train_x_Morgan2comboSMOTE.csv', index=False, header=False)
# trainydfcomboSMOTE.to_csv(split_path+'inhib_train_y_Morgan2comboSMOTE.csv', index=False, header=False)
# y_test_df.to_csv(split_path+'inhib_test_y_Morgan2comboSMOTE.csv', index=False, header=False)
# test_x_comboSMOTE.to_csv(split_path+'inhib_test_x_Morgan2comboSMOTE.csv', index=False, header=False)
trainXdfcomboSMOTE.shape, trainydfcomboSMOTE.shape, test_x_comboSMOTE.shape, y_test.shape

# Combo r=2, UNDER

In [ ]:
undersampler = RandomUnderSampler()
trainX_allUNDER, trainy_allUNDER = undersampler.fit_resample(X_train_allfeats, y_train) 
trainXallUNDERdf = pd.DataFrame(trainX_allUNDER)
trainyallUNDERdf = pd.DataFrame(trainy_allUNDER)
testxdf = pd.DataFrame(X_test_allfeats)
testydf = pd.DataFrame(y_testmorgan) 

# trainXallUNDERdf.to_csv(split_path+'inhib_train_x_Morgan2comboUNDER.csv', index=False, header=False)
# trainyallUNDERdf.to_csv(split_path+'inhib_train_y_Morgan2comboUNDER.csv', index=False, header=False)
# testydf.to_csv(split_path+'inhib_test_y_Morgan2comboUNDER.csv', index=False, header=False)
# testxdf.to_csv(split_path+'inhib_test_x_Morgan2comboUNDER.csv', index=False, header=False)

trainXallUNDERdf.shape, trainyallUNDERdf.shape, testydf.shape, testxdf.shape

# Combo r=2, ADASYN

In [ ]:
adasyn = ADASYN() 
trainX_allADASYN, trainy_allADASYN = adasyn.fit_resample(X_train_allfeats, y_train)
trainXallADASYNdf = pd.DataFrame(trainX_allADASYN)
trainyallADASYNdf = pd.DataFrame(trainy_allADASYN)
testxdfADASYN = pd.DataFrame(X_test_allfeats)
testydfADASYN = pd.DataFrame(y_testmorgan) 

# trainXallADASYNdf.to_csv(split_path+'inhib_train_x_Morgan2comboADASYN.csv', index=False, header=False)
# trainyallADASYNdf.to_csv(split_path+'inhib_train_y_Morgan2comboADASYN.csv', index=False, header=False)
# testxdfADASYN.to_csv(split_path+'inhib_test_y_Morgan2comboADASYN.csv', index=False, header=False)
# testydfADASYN.to_csv(split_path+'inhib_test_x_Morgan2comboADASYN.csv', index=False, header=False)

trainXallADASYNdf.shape, trainyallADASYNdf.shape, testxdfADASYN.shape, testydfADASYN.shape